In [2]:
import difflib

s1 = "abcdef"
s2 = "azcedf"

matcher = difflib.SequenceMatcher(None, s1, s2)

In [3]:
from difflib import SequenceMatcher
from rich.text import Text
import rich

from difflib import SequenceMatcher
from rich.console import Console
from rich.text import Text

from difflib import SequenceMatcher
from rich.console import Console
from rich.text import Text

def highlight_differences(prompt_a: str, prompt_b: str):
    matcher = SequenceMatcher(None, prompt_a, prompt_b)
    text = Text()

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            text.append(prompt_a[i1:i2])
        elif tag == 'replace':
            # Strike with colored line, default text color
            text.append(Text(prompt_a[i1:i2], style="strike color(#ff0000)"))
            text.append(Text(prompt_b[j1:j2], style="green"))
        elif tag == 'delete':
            text.append(Text(prompt_a[i1:i2], style="strike color(#ff0000)"))
        elif tag == 'insert':
            text.append(Text(prompt_b[j1:j2], style="green"))

    console = Console()
    console.print(text)


# Example usage
s1 = "abcdef"
s2 = "azcedf"
highlight_differences("hello beautiful world", "hello brave world")


hello braveautiful world

In [4]:
from difflib import SequenceMatcher
from IPython.display import display, HTML

color_palette = {
    "delete": "#FF90BB",
    "insert": "#7cd9e6",
    "replace": "#FF90BB",
    "equal": "#8ACCD5",
}

def highlight_differences_html(prompt_a: str,
                               prompt_b: str,
                               color_palette=color_palette) -> None:

    delete_color = color_palette["delete"]
    insert_color = color_palette["insert"]

    matcher = SequenceMatcher(None, prompt_a, prompt_b)
    html = ""

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            html += prompt_a[i1:i2]
        elif tag == 'replace':
            html += (
                f"<span style='color: {delete_color}; text-decoration: line-through; "
                f"text-decoration-color: {delete_color};'>{prompt_a[i1:i2]}</span>"
            )
            html += f"<span style='color: {insert_color};'>{prompt_b[j1:j2]}</span>"
        elif tag == 'delete':
            html += (
                f"<span style='color: {delete_color}; text-decoration: line-through; "
                f"text-decoration-color: {delete_color};'>{prompt_a[i1:i2]}</span>"
            )
        elif tag == 'insert':
            html += f"<span style='color: {insert_color};'>{prompt_b[j1:j2]}</span>"
    display(HTML(f"<div>{prompt_a}</div>"))
    display(HTML(f"<div>{html}</div>"))


highlight_differences_html("hello beautiful world", "hello brave world")

In [5]:
from difflib import SequenceMatcher
from rich.console import Console
from rich.text import Text
import re

from cheatsheet import CHEATSHEETS
from prompt import PROMPTS
from IPython import get_ipython

color_palette = {
    "delete": "#FF90BB",
    "insert": "#7CD9E6",
    "equal": "#A3A3A3",
    "f-string expression": "#87A2FF",
    "separator": "#86D293",
}

def highlight_fstring_parts(s, default_style, expr_style):
    text = Text()
    pattern = re.compile(r"({[^{}]+})")
    last_end = 0

    for match in pattern.finditer(s):
        start, end = match.span()
        # Add preceding regular text
        if start > last_end:
            text.append(s[last_end:start], style=default_style)
        # Add expression with special style
        text.append(s[start:end], style=expr_style)
        last_end = end

    # Add any remaining text after last match
    if last_end < len(s):
        text.append(s[last_end:], style=default_style)

    return text

def highlight_differences(prompt_a: str, prompt_b: str, color_palette=color_palette, print_separator=False):
    color_delete = color_palette["delete"]
    color_insert = color_palette["insert"]
    color_equal = color_palette["equal"]
    color_expression = color_palette["f-string expression"]
    color_separator = color_palette["separator"]

    matcher = SequenceMatcher(None, prompt_a, prompt_b)

    # Init rich.Text() object
    text_prompt_a = Text()
    text_prompt_b = Text()

    text_prompt_a.append(Text("- ", style=f"bold {color_delete}"))
    text_prompt_a.append(highlight_fstring_parts(prompt_a, color_equal, color_expression))
    text_prompt_b.append(Text("+ ", style=f"bold {color_insert}"))

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            text_prompt_b.append(
                highlight_fstring_parts(prompt_a[i1:i2], color_equal, color_expression))
        elif tag == 'replace':
            # Strike with colored line, default text color
            text_prompt_b.append(Text(prompt_a[i1:i2], style=f"strike {color_delete}"))
            text_prompt_b.append(Text(prompt_b[j1:j2], style=f"{color_insert}"))
        elif tag == 'delete':
            text_prompt_b.append(Text(prompt_a[i1:i2], style=f"strike {color_delete}"))
        elif tag == 'insert':
            text_prompt_b.append(Text(prompt_b[j1:j2], style=f"{color_insert}"))

    console = Console()
    separator = Text("-------------------------------------", style=color_separator)
    console.print(text_prompt_a)
    if print_separator:
        console.print(separator)
    console.print(text_prompt_b)
    

system_prompt_a = PROMPTS["entity_extraction"]
system_prompt_b = CHEATSHEETS["fill_nightly"]


# Example usage
highlight_differences("hello beautiful world", "hello brave world", print_separator=False)

- hello beautiful world

+ hello braveautiful world

In [6]:
from pretty_prompt_compare import pretty_prompt_compare
pretty_prompt_compare("hello beautiful world", "hello brave world")

- hello beautiful world

+ hello braveautiful world

In [7]:
from rich.text import Text
from rich.console import Console

def _highlight_focus_parts(prompt: str,
                           default_style: str,
                           focus_style: str,
                           focus_list: list[str]) -> Text:
    text = Text(prompt, style=default_style)

    for focus in focus_list:
        start = 0
        while True:
            index = prompt.find(focus, start)
            if index == -1:
                break
            text.stylize(focus_style, index, index + len(focus))
            start = index + len(focus)

    return text

# ----------------------
# 🎮 Try it out
# ----------------------

console = Console()

# Test inputs
prompt_1 = "The quick brown fox jumps over the lazy fox"
focus_1 = ["quick", "fox", "laz", "lazy"]
highlighted_1 = _highlight_focus_parts(prompt_1, "white", "bold red", focus_1)
console.rule("[bold yellow]Test 1: Basic Focus")
console.print(highlighted_1)

prompt_2 = "Hello world! This world is full of wonders. Hello again!"
focus_2 = ["Hello", "world"]
highlighted_2 = _highlight_focus_parts(prompt_2, "green", "underline bold magenta", focus_2)
console.rule("[bold yellow]Test 2: Multiple Occurrences")
console.print(highlighted_2)

prompt_3 = "CASE sensitivity Check: case Case CASE"
focus_3 = ["case"]
highlighted_3 = _highlight_focus_parts(prompt_3, "cyan", "reverse red", focus_3)
console.rule("[bold yellow]Test 3: Case-Sensitive")
console.print(highlighted_3)

# Modify these variables to experiment!
your_prompt = "Change this prompt and try your own focus words!"
your_focus = ["Change", "your", "focus"]
highlighted_yours = _highlight_focus_parts(your_prompt, "dim", "bold yellow", your_focus)
console.rule("[bold yellow]Test 4: Your Turn")
console.print(highlighted_yours)


─────────────────────────────────────────────── Test 1: Basic Focus ───────────────────────────────────────────────

The quick brown fox jumps over the lazy fox

────────────────────────────────────────── Test 2: Multiple Occurrences ───────────────────────────────────────────

Hello world! This world is full of wonders. Hello again!

───────────────────────────────────────────── Test 3: Case-Sensitive ──────────────────────────────────────────────

CASE sensitivity Check: case Case CASE

──────────────────────────────────────────────── Test 4: Your Turn ────────────────────────────────────────────────

Change this prompt and try your own focus words!

In [ ]:
from pretty_prompt_compare import PrettyCompare

pretty_compare = PrettyCompare(compare_response=True, target=["brave", "beautiful", "world"])

# pretty_compare = PrettyCompare(compare_response=True, print_separator=True)
"Hello beautiful {world}" |pretty_compare| "Hello brave {world}"

- Hello beautiful {world}

+ Hello braveautiful {world}

In [1]:
from pretty_prompt_compare import pretty_response_compare

pretty_response_compare("Hello brave world", "Hello beautiful world", ["brave", "beautiful"])

- Hello brave world

+ Hello breaveutiful world

In [ ]:
# turn a python string into a rich.Text object
from rich.text import Text
from rich.console import Console

console = Console()

text = Text("Hello world!")
text.stylize("bold red", 0, 5)

type(text[0:6])

# a_text = Text("Hello world!")
# a_text.append(text)

rich.text.Text